In [ ]:
!pip install konlpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 라이브러리 다운로드

In [ ]:
!pip install torchtext==0.4.0

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchtext import data, datasets
from torchtext.vocab import GloVe

import re
import os
import math
import random
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from collections import Counter
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
torch.manual_seed(777)
random.seed(777)
np.random.seed(777)

## Data setting

In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
pre = pd.read_csv("/content/drive/My Drive/final_data_prepro.csv"); pre

,Unnamed: 0,product_names,categories,retypes,rvs,meta_sizes,meta_brights,meta_colors,meta_thicks,scores,helpfuls,uploadAts,collectAts,cus_sex,cus_height,cus_weight
0,0,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,길어서 후드 밑에 받쳐입어야할 것 같아요\n약간 가성비..?! 두꺼워서 가을에입기 ...,커요,보통이에요,보통이에요,두꺼워요,6,0,3시간 전,2021-11-04 23:35:35.399930,여성,161.0,57.0
1,1,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,오버핏으로 레이어드 해서 입는걸 좋아하는데\n다른 코튼 티셔츠들보다 목부분이 튼튼하...,커요,보통이에요,선명해요,두꺼워요,10,0,1일 전,2021-11-04 23:35:35.401920,여성,158.0,50.0
2,2,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,길이감이 좋아서 어디 받쳐입기 굉장히 좋아요 그리고 적당히 두꺼워서 맨살이 안 비쳐요!,보통이에요,보통이에요,보통이에요,보통이에요,8,0,2일 전,2021-11-04 23:35:35.402919,남성,178.0,68.0
3,3,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,길이감 좋고 두께도 적당해서 레이어드 하기 좋아요!,보통이에요,보통이에요,보통이에요,보통이에요,8,0,2일 전,2021-11-04 23:35:35.404927,남성,178.0,68.0
4,4,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,사이즈잘맞아요 핏도좋습니다 감사합니다 잘입을게요,보통이에요,보통이에요,보통이에요,보통이에요,10,0,3일 전,2021-11-04 23:35:35.405932,남성,175.0,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53995,23995,우먼즈 라운지 쇼츠 [코럴],숏 팬츠,스타일 후기,딱 이쁜 짧은 반바지예요 아무때나 입고 나가기 좋은... 저랑 키 같으면 s사세여,보통이에요,보통이에요,보통이에요,보통이에요,10,0,8일 전,2021-05-31 22:37:43.744457,여성,162.0,50.0
53996,23996,우먼즈 라운지 쇼츠 [코럴],숏 팬츠,스타일 후기,바지 실제로 입어보니까 생각보다 엄청 짧아서 놀랐어요,작아요,보통이에요,보통이에요,보통이에요,8,5,9일 전,2021-05-31 22:37:43.745456,여성,159.0,47.0
53997,23997,우먼즈 라운지 쇼츠 [코럴],숏 팬츠,스타일 후기,집에서 입으려고 샀어요\n잠옷 보다는 두께가 있는데 운동복정도는 아니에요,보통이에요,어두워요,선명해요,보통이에요,10,0,12일 전,2021-05-31 22:37:43.747455,여성,158.0,58.0
53998,23998,우먼즈 라운지 쇼츠 [코럴],숏 팬츠,스타일 후기,여름에 입기 딱 좋을 듯 시원하고 핏도 잘 나왔어요\n편해서 잘 입을 것 같습니다.,보통이에요,어두워요,선명해요,보통이에요,10,1,12일 전,2021-05-31 22:37:43.748456,여성,154.0,43.0


In [ ]:
train = pd.read_csv("/content/drive/My Drive/crawling_20211104.csv")

In [ ]:
df_shuffled=train.iloc[np.random.permutation(train.index)].reset_index(drop=True)

In [ ]:
torch.manual_seed(777)
random.seed(777)
np.random.seed(777)

# train = pd.read_csv("/content/drive/My Drive/crawling_20211104.csv")

train = df_shuffled.copy()

label = []
for s in train['scores']:
  if s == "width: 100%":
    label.append(0)
  elif s == "width: 80%":
    label.append(1)
  else:
    label.append(2)

train['label'] = label

train_df = train[:-8000]
val_df = train[-8000:]

total_tokens = [okt.morphs(sentence) for sentence in tqdm(train['rvs'])]

stopwords = ['의','가','이','은','들','는','과','도','를','으로','자','에','와','한','하다','대다','년','월','대']

100%|██████████| 30000/30000 [02:42<00:00, 185.14it/s]


In [ ]:
using_data = train[['rvs', 'label', 'retypes', 'categories', 'meta_sizes', 'meta_brights', 'meta_colors', 'meta_thicks']]

In [ ]:
meta_df = pd.DataFrame()
for c in using_data.columns.tolist()[2:]:
  dummy_cate = pd.get_dummies(using_data[c])
  meta_df = pd.concat([meta_df, dummy_cate], axis=1)

train_meta_df = meta_df[:-8000]
val_meta_df = meta_df[-8000:]

### 단어 집합 만들기 with GloVe (torchtext)

In [ ]:
# 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=okt.morphs,
                  lower=True,
                  batch_first=True,
                  stop_words = stopwords,
                  fix_length=100)

LABEL = data.LabelField(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

TEXT.build_vocab(total_tokens,  vectors=GloVe(name='6B', dim=200), min_freq=3, max_size=10000)
LABEL.build_vocab(['neg','pos'])

vocab = TEXT.vocab
print('단어 집합의 크기 : {}'.format(len(vocab)))
vocab_size = len(vocab)

word_dict = TEXT.vocab.stoi
rev_word_dict = {v:k for k, v in word_dict.items()}

.vector_cache/glove.6B.zip: 862MB [05:11, 2.77MB/s]                           
100%|█████████▉| 399999/400000 [00:31<00:00, 12901.16it/s]


단어 집합의 크기 : 8949


In [ ]:
using_train_data = train_df[['rvs', 'label', 'categories', 'retypes',
       'meta_sizes', 'meta_brights', 'meta_colors', 'meta_thicks', ]]
      #  'pur_option', 'cus_sex', 'cus_height', 'cus_weight']]

using_val_data = val_df[['rvs', 'label', 'categories', 'retypes',
       'meta_sizes', 'meta_brights', 'meta_colors', 'meta_thicks', ]]
      #  'pur_option', 'cus_sex', 'cus_height', 'cus_weight']]

In [ ]:
 class BinaryDataset(Dataset):

    def __init__(self, df, meta_df, word_dict, min_len, only_ct = True, test=False):
        self.df = df
        self.meta_df = meta_df
        self.w2i = word_dict
        self.i2w = {v:k for k, v in word_dict.items()}
        self.min_len = min_len
        self.test = test
        self.only_ct = only_ct

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sentence = self.df.iloc[idx, 0]
        label = self.df.iloc[idx, 1]

        if self.only_ct:
          meta = self.meta_df.iloc[idx, :]
        else:
          meta = self.meta_df.iloc[idx, 3:9]

        text = [tok for tok in word_tokenize(sentence.lower())]
        if len(text) < self.min_len:
            text += [TEXT.pad_token] * (self.min_len - len(text))
        else :
            text = text[:self.min_len]
        indexed = torch.tensor([self.w2i[t] for t in text])

        return indexed, torch.tensor(label), torch.tensor(meta).float()

BATCH = 16

train_data = BinaryDataset(using_train_data, train_meta_df, word_dict, 64)#, only_ct = False)
train_loader = DataLoader(train_data, batch_size=BATCH, shuffle=True)

val_data = BinaryDataset(using_val_data, val_meta_df, word_dict, 64)#, only_ct = False)
eval_loader = DataLoader(val_data, batch_size=BATCH, shuffle=False)

print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('검증 샘플의 개수 : {}'.format(len(val_data)))

훈련 샘플의 개수 : 22000
검증 샘플의 개수 : 8000


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_layers, hidden_dim, output_dim, meta_input_dim, meta_hidden_dim,
                 batch_size, pad_idx, use_m=True, dropout=0.2):
        super().__init__()
        self.use_m = use_m
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        self.meta_feature = nn.Sequential(
            nn.Linear(meta_input_dim, meta_hidden_dim),
            nn.Linear(meta_hidden_dim, meta_input_dim)
        )

        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first = True,
            # bidirectional = True
        )
        if self.use_m:
            
          self.classifier = nn.Sequential(
              nn.Linear(hidden_dim + meta_input_dim, hidden_dim),
              nn.Linear(hidden_dim, output_dim)
          )
        else :
          self.classifier = nn.Sequential(
              nn.Linear(hidden_dim, hidden_dim),
              nn.Linear(hidden_dim, output_dim)
          )
          

    def forward(self, text, meta):
        embedded = self.embedding(text)
        out, (ht, ct) = self.lstm(embedded)
        
        if self.use_m :
          meta_f = self.meta_feature(meta)
          cat = torch.cat([ht[-1], meta_f], dim=1)
          out = self.classifier(cat)

        else:
          out = self.classifier(ht[-1])

        return out
    
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))

In [ ]:
class GRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_layers, hidden_dim, output_dim, 
                 meta_input_dim, meta_hidden_dim,
                 batch_size, pad_idx, use_m=True, dropout=0.2):
        super(GRU, self).__init__()

        self.use_m = use_m
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.meta_feature = nn.Sequential(
            nn.Linear(meta_input_dim, meta_hidden_dim),
            nn.Linear(meta_hidden_dim, meta_input_dim)
        )

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers, dropout=dropout, batch_first=True)
        if self.use_m:
            
          self.classifier = nn.Sequential(
              nn.Linear(hidden_dim + meta_input_dim, hidden_dim),
              nn.Linear(hidden_dim, output_dim)
          )
        else :
          self.classifier = nn.Sequential(
              nn.Linear(hidden_dim, hidden_dim),
              nn.Linear(hidden_dim, output_dim)
          )

    def forward(self, text, meta):
        embedded = self.embedding(text)
        out, gt = self.gru(embedded)

        if self.use_m:
          meta_f = self.meta_feature(meta)
          cat = torch.cat([out[:,-1], meta_f], dim=1)
          out = self.classifier(cat)
          
        else :
          out = self.classifier(out[:,-1])

        return out
    
    def init_hidden(self):
        return autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim))

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 200
NUM_LAYERS = 1
BATCH_SIZE = BATCH
HIDDEN_DIM = 128
OUTPUT_DIM = 3
DROPOUT = 0.3
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

META_INPUT = 21   ## 제품 카테고리만 하면 6
META_HIDDEN = 64

# model = LSTM(INPUT_DIM, EMBEDDING_DIM, NUM_LAYERS, HIDDEN_DIM, OUTPUT_DIM, META_INPUT, META_HIDDEN, BATCH_SIZE, PAD_IDX, DROPOUT)
model = GRU(INPUT_DIM, EMBEDDING_DIM, NUM_LAYERS, HIDDEN_DIM, OUTPUT_DIM, META_INPUT, META_HIDDEN, BATCH_SIZE, PAD_IDX, DROPOUT)

pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,935,025 trainable parameters


In [ ]:
def binary_accuracy(preds, y):

    output_softmax = torch.log_softmax(preds, 1)
    _, output_tags = torch.max(output_softmax, 1)
    correct_pred = (output_tags == y).float()
    acc = correct_pred.sum() / len(correct_pred)
    # acc_sum += acc * len(correct_pred)
    return acc, len(correct_pred)

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    acc_sum = 0

    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, label, meta = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        outputs = model(text, meta)
        predictions = F.softmax(outputs, dim=1)
        loss = criterion(predictions, label)
        
        acc, leng = binary_accuracy(outputs, label)
        acc_sum += acc * leng
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, label, meta = batch[0].to(device), batch[1].to(device), batch[2].to(device)

            outputs = model(text, meta)#.squeeze(1)
            predictions = F.softmax(outputs, dim=1)

            loss = criterion(predictions, label)
            
            acc, _ = binary_accuracy(outputs, label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, eval_loader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'lstm_model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')